# Agentic AI

## Prerequisites

### Install a Local LLM with Ollama

To run this project locally, we will install and use **Ollama**, a lightweight runtime for local large language models.

**Download Ollama:**  
https://ollama.com/

Once installed, you can pull any model you want to run.  
Below are a few recommended examples, but you are free to pick any size or model from the Ollama library.

ollama pull qwen3:0.6b

or

ollama pull ibm/granite4:350m

or

Choose any model you prefer, make sure the model supports tools.
Browse available models here:
https://ollama.com/library



### Python requirements

In [1]:
!pip install langgraph langchain-google-genai langchain-core mcp langchain-ollama

## 1. Define FastMCP Tools

In [2]:
from color_blocks_state import color_blocks_state, init_goal_for_search
from heuristics import init_goal_for_heuristics, advanced_heuristic
from search import search


In [3]:
from mcp.server.fastmcp import FastMCP
import math

# Initialize FastMCP
mcp = FastMCP("Unified Solver")

@mcp.tool()
def calculate_sum(a: float, b: float) -> float:
    """Calculates the sum of two numbers."""
    return a + b

@mcp.tool()
def calculate_power(base: float, exponent: float) -> float:
    """Calculates the power of a base number."""
    return math.pow(base, exponent)

# TO DO: Add more tools as needed for your application
@mcp.tool()
def solve_search_cost(start_state: str, goal_state: str) -> int:
    """Compute the optimal solution cost. 
    start_state: string like '(5,2),(1,3),(9,22),(21,4)'
    goal_state: string like '2,22,4,3' (ONLY numbers and commas)"""
   
    #removal of chars that the model seems to add
    clean_goal = goal_state.replace("(", "").replace(")", "").replace(" ", "")
    
    init_goal_for_search(clean_goal)
    init_goal_for_heuristics(clean_goal)
    start = color_blocks_state(start_state)
    path = search(start, advanced_heuristic)

    if path is None: return -1
    return len(path) - 1


In [4]:
print(solve_search_cost("(5,2),(1,3),(9,22),(21,4)", "2,22,4,3"))


6


## 2. LLM + MCP

### 2.1. Global instance of our LLM

In [5]:
import os
from langchain_ollama import ChatOllama
#from langchain_google_genai import ChatGoogleGenerativeAI


# Choose your model here, can be Ollama or Google Gemini
# Can also switch between different model sizes as needed
# model = "qwen3:0.6b"
#model = "ibm/granite4:350m"
model = "llama3.2:3b"
global_llm = ChatOllama(model=model, temperature=0.0)

# SETUP API KEY if using Google Gemini
#os.environ["GOOGLE_API_KEY"] = "YOUR_GOOGLE_API_KEY_HERE"

# model = "gemini-2.5-flash"
# model = "gemini-2.5-flash-lite"
# global_llm = ChatGoogleGenerativeAI(model=model, temperature=0)


None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


### 2.2. Our agent graph

In [6]:
from langgraph.graph import MessagesState, START, StateGraph
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver # Optional: For saving graph state
import uuid


def create_agent_graph(sys_msg, tools):
    """ Creates a LangGraph StateGraph with the given tools integrated."""

    llm = global_llm

    if tools:
        llm_with_tools = llm.bind_tools(tools)
    else:
        llm_with_tools = llm

    # Node
    def assistant(state: MessagesState):
        return {
            "messages": [
                llm_with_tools.invoke([sys_msg] + state["messages"], think=False)
            ]
        }

    # Graph
    builder = StateGraph(MessagesState)

    # Define the basic graph structure
    builder.add_node("assistant", assistant)
    builder.add_edge(START, "assistant")

    if tools:
        builder.add_node("tools", ToolNode(tools))  
        builder.add_conditional_edges(
            "assistant",
            tools_condition,
        )
        builder.add_edge("tools", "assistant")

    react_graph = builder.compile()

    return react_graph


async def run_agent(prompt, tools, sys_msg=""):

    sys_msg = SystemMessage(content=sys_msg)

    # 3. Create Graph
    graph = create_agent_graph(sys_msg, tools)
    
    unique_id = str(uuid.uuid4())
    # 4. Run (using ainvoke for async tools)
    config = {"configurable": {"thread_id": unique_id}}
    result = await graph.ainvoke({"messages": [HumanMessage(content=prompt)]}, config)

    last_msg = result["messages"][-1].content

    # Extract tool names and outputs
    tools_used = []
    tools_output = []
    
    # Parsing logic specific to your request
    for msg in result["messages"]:
        # In LangChain, tool calls are usually in 'tool_calls' attribute of AIMessage
        # or 'name' attribute if it is a ToolMessage
        if hasattr(msg, 'tool_calls') and msg.tool_calls:
             for tool_call in msg.tool_calls:
                tools_used.append(tool_call['name'])
        
        if msg.type == 'tool':
            tools_output.append(msg.content)

    return last_msg, tools_used, tools_output

### 2.3. Tools that run spacific agent (with tools and without)

In [7]:

@mcp.tool()
async def ask_agent_with_tools(prompt: str) -> str: # specifically return str to avoid model mistakes
    """ Runs the agent with access to tools. Expects a single string prompt. """
    tools = [solve_search_cost]
    results = await run_agent(prompt, tools)
    return results[0]

@mcp.tool()
async def ask_agent_without_tools(prompt: str) -> str:  # specifically return str to avoid model mistakes
    """ Runs the agent without access to tools. Expects a single string prompt. """
    results = await run_agent(prompt, [])
    return results[0]

## 3. Run the Test

In [8]:
sys_msg = """
    You are a Research Supervisor. You must evaluate two assistants on a Color-Blocks search problem.
    
    CRITICAL: When calling the tools, you MUST pass the full problem description including:
    Start state: "(5,2),(1,3),(9,22),(21,4)"
    Goal state: "2,22,4,3"

    OUTPUT FORMAT RULES:
    You MUST provide your final response in this exact template:
    
    **Agent with Tools Output:** [Insert their cost here]
    **Agent without Tools Output:** [Insert their cost here]
    
    **Comparison:** [Explain why the costs differ. Note that the agent with tools uses an optimal A* algorithm, while the other estimates.]
"""

prompt = """
    Compare the two assistants on this Color-Blocks search question:
    Start state: "(5,2),(1,3),(9,22),(21,4)"
    Goal state: "2,22,4,3"
    Question: What is the optimal solution cost (minimum number of moves) from start to goal?
"""

tool_list = [ask_agent_with_tools, ask_agent_without_tools]

response, tools, outputs = await run_agent(prompt, tool_list, sys_msg)
print(f"Response: {response}")
print(f"Tools Used: {tools}")
print(f"Tool Outputs: {outputs}")


HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


Response: **Agent with Tools Output:** 6
**Agent without Tools Output:** 10
**Comparison:** The agent with tools uses an optimal A* algorithm, while the other estimates. The agent with tools found the minimum number of moves required to transform the start state into the goal state, which is 6. In contrast, the agent without tools estimated a higher cost of 10.
Tools Used: ['ask_agent_with_tools', 'ask_agent_without_tools']
Tool Outputs: ['The optimal solution cost (minimum number of moves) from start to goal is 6.', 'To find the minimum number of moves required to transform the start state into the goal state, we can use a breadth-first search (BFS) algorithm.\n\nThe start state is: "(5,2), (1,3), (9,22), (21,4)"\n\nThe goal state is: "2, 22, 4, 3"\n\nWe will transform each pair of coordinates into the desired order by swapping them. We can use a queue to keep track of the states to be explored.\n\nHere\'s the BFS algorithm:\n\n1. Start with the initial state: "(5,2), (1,3), (9,22), (